In [1]:
import os
import pandas as pd
import numpy
import requests
import json
import matplotlib.style as style
from sqlalchemy import create_engine
from pandas.io.json import json_normalize

https://github.com/xmunoz/sodapy
https://dev.socrata.com/docs/queries/
https://dev.socrata.com/foundry/data.cityofnewyork.us/43nn-pn8j
https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j

In [4]:
health_df = pd.read_csv('data/NYC_Restaurants.csv')

In [5]:
health_df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,6/11/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,6,A,6/11/2019,6/21/2019,Cycle Inspection / Re-inspection
3,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8,A,6/6/2019,6/21/2019,Cycle Inspection / Initial Inspection
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,6/6/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,8,A,6/6/2019,6/21/2019,Cycle Inspection / Initial Inspection


In [6]:
health_df.describe()

,CAMIS,ZIPCODE,SCORE
count,6.514000e+03,6332.000000,6514.000000
mean,4.632415e+07,10705.445831,17.503070
std,4.392160e+06,591.097305,12.222303
min,3.007544e+07,10001.000000,0.000000
25%,4.142820e+07,10026.000000,10.000000
50%,5.001494e+07,11101.000000,13.000000
75%,5.006358e+07,11229.000000,24.000000
max,5.009364e+07,11694.000000,91.000000


In [7]:
#new_results_df= results_df[results_df["grade_date"] == ] 

In [8]:
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514 entries, 0 to 6513
Data columns (total 18 columns):
CAMIS                    6514 non-null int64
DBA                      6514 non-null object
BORO                     6514 non-null object
BUILDING                 6503 non-null object
STREET                   6514 non-null object
ZIPCODE                  6332 non-null float64
PHONE                    6514 non-null object
CUISINE DESCRIPTION      6514 non-null object
INSPECTION DATE          6514 non-null object
ACTION                   6514 non-null object
VIOLATION CODE           6469 non-null object
VIOLATION DESCRIPTION    6363 non-null object
CRITICAL FLAG            6514 non-null object
SCORE                    6514 non-null int64
GRADE                    6514 non-null object
GRADE DATE               6514 non-null object
RECORD DATE              6514 non-null object
INSPECTION TYPE          6514 non-null object
dtypes: float64(1), int64(2), object(15)
memory usage: 916.1+ KB


In [9]:
#converting all the dates to datetime to make easy for processing
health_df["GRADE DATE"]=pd.to_datetime(health_df["GRADE DATE"])
health_df["INSPECTION DATE"]=pd.to_datetime(health_df["INSPECTION DATE"])
health_df["RECORD DATE"]=pd.to_datetime(health_df["RECORD DATE"])

In [10]:
health_df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,6,A,2019-06-11,2019-06-21,Cycle Inspection / Re-inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,6,A,2019-06-11,2019-06-21,Cycle Inspection / Re-inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,2019-06-11,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,6,A,2019-06-11,2019-06-21,Cycle Inspection / Re-inspection
3,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2019-06-06,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,8,A,2019-06-06,2019-06-21,Cycle Inspection / Initial Inspection
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019.0,2122452912,Irish,2019-06-06,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,8,A,2019-06-06,2019-06-21,Cycle Inspection / Initial Inspection


In [11]:
health_df.shape

(6514, 18)

In [12]:
# Try some group by for new aggregate table
# school_group_df = school_data_complete.groupby(["school_name"]).agg({"student_name": "count","budget":"mean"})

# school_group_df = merged_df.rename(columns={"student_name":"Total Students","budget_x":"Total School Budget","type":"School Type","math_score":"Average Math Score","reading_score":"Average Reading Score","type":"School Type"})

#reduce the final output of columns in new dataframe, but keep school_group_df because it is used further down
#drop size_y, budget_y, budget_y
#make school_name the index for the final dataframe
# school_summary = school_group_df.drop(columns=["size_x","size_y","budget_y","Passing Math Students","Passing Reading Students","School ID"])
# school_summary = school_summary[["school_name","School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
# school_summary = school_summary.set_index("school_name")
# # remove school name header
# school_summary.index.name = None
# final_math_df = final_math_df.rename(columns={"avg 9th grade math":"9th","avg 10th grade math":"10th"})

In [13]:
# Violation counts by Restaurant name and ID
violaton_count_df = health_df.groupby(["CAMIS"]).agg({"DBA":"first","VIOLATION CODE":"count"})
# violaton_count_df.DBA = violaton_count_df.DBA.astype(int)
violaton_count_df = violaton_count_df.rename(columns={"VIOLATION CODE":"Count of Violations","DBA":"Restaurant Name"})
violaton_count_df.index.name = "Restaurant ID"
violaton_count_df.head(10)

,Restaurant Name,Count of Violations
Restaurant ID,,
30075445,MORRIS PARK BAKE SHOP,3
30191841,DJ REYNOLDS PUB AND RESTAURANT,2
40362715,THE COUNTRY CAFE,3
40362869,SHASHEMENE INT'L RESTAURANT,2
40364286,PLAZA BAGELS & DELI,2
40364404,POLISH NATIONAL HOME,3
40364956,THE NEW STARLING ATHLETIC CLUB OF THE BRONX,1
40365043,SHELL LANES,2
40365098,FORLINIS RESTAURANT,4


In [14]:
violaton_count_df.info()
# 2354 unique restaurants

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2354 entries, 30075445 to 50093639
Data columns (total 2 columns):
Restaurant Name        2354 non-null object
Count of Violations    2354 non-null int64
dtypes: int64(1), object(1)
memory usage: 55.2+ KB


In [15]:
violaton_count_df.to_csv('Nyc_Resto_violations.csv',index=False)

In [45]:
health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514 entries, 0 to 6513
Data columns (total 18 columns):
CAMIS                    6514 non-null int64
DBA                      6514 non-null object
BORO                     6514 non-null object
BUILDING                 6503 non-null object
STREET                   6514 non-null object
ZIPCODE                  6332 non-null float64
PHONE                    6514 non-null object
CUISINE DESCRIPTION      6514 non-null object
INSPECTION DATE          6514 non-null datetime64[ns]
ACTION                   6514 non-null object
VIOLATION CODE           6469 non-null object
VIOLATION DESCRIPTION    6363 non-null object
CRITICAL FLAG            6514 non-null object
SCORE                    6514 non-null int64
GRADE                    6514 non-null object
GRADE DATE               6514 non-null datetime64[ns]
RECORD DATE              6514 non-null datetime64[ns]
INSPECTION TYPE          6514 non-null object
dtypes: datetime64[ns](3), float64(1), int

In [59]:
# Violation counts by Borough and Restaurant
violation_by_boro = health_df.groupby(["BORO","DBA"]).agg({"VIOLATION CODE":"count"})
violation_by_boro = violation_by_boro.rename(columns={"VIOLATION CODE":"Count of Violations"})
violation_by_boro.head()

Count of Violations
BORO  DBA                                                                 
BRONX 3-J RESTAURANT AND PIZZA                                           2
      619 BAR & RESTAURANT EL SALVADORENO                                3
      AL CHOLO WEST INDIAN BAKERY & HEALTH FOOD STORE                    1
      ALBERT'S COFFEE SHOP                                               2
      ANTHONY'S AND SALS PIZZA EXPRESS                                   3

In [64]:
# Violation counts by cuisine description
violation_by_cuisine = health_df.groupby(["CUISINE DESCRIPTION"]).agg({"VIOLATION CODE":"count"})
violation_by_cuisine = violation_by_cuisine.rename(columns={"VIOLATION CODE":"Count of Violations"})
violation_by_cuisine.head(25)

,Count of Violations
CUISINE DESCRIPTION,
Afghan,7
African,12
American,1407
Armenian,3
Asian,86
Australian,3
Bagels/Pretzels,46
Bakery,176
Bangladeshi,8
